In [331]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import itertools

In [332]:
bike_dataset = pd.read_csv('/content/drive/MyDrive/hour.csv') # Load the dataset

X = bike_dataset.drop(columns=['dteday', 'cnt'])
Y = bike_dataset['cnt'] # The target variable
print(X.head())
print(Y.head())

   season  yr  mnth  hr  holiday  weekday  workingday  weathersit  temp  \
0       1   0     1   0        0        6           0           1  0.24   
1       1   0     1   1        0        6           0           1  0.22   
2       1   0     1   2        0        6           0           1  0.22   
3       1   0     1   3        0        6           0           1  0.24   
4       1   0     1   4        0        6           0           1  0.24   

    atemp   hum  windspeed  
0  0.2879  0.81        0.0  
1  0.2727  0.80        0.0  
2  0.2727  0.80        0.0  
3  0.2879  0.75        0.0  
4  0.2879  0.75        0.0  
0    16
1    40
2    32
3    13
4     1
Name: cnt, dtype: int64


In [333]:
X_test, X_temp, Y_test, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # Splitting the dataset
X_val, X_train, Y_val, Y_train = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [334]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
      # Initialize hyperparameters for the tree
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    class Node:
        __slots__ = ('val', 'feat', 'thresh', 'left', 'right')
        def __init__(self, val=None, feat=None, thresh=None, left=None, right=None):
          # Node stores either a value (leaf) or a splitting feature and threshold
            self.val = val
            self.feat = feat      # Feature index for splitting
            self.thresh = thresh  # Threshold value for splitting
            self.left = left
            self.right = right

    def fit(self, X, Y):
        X, Y = np.array(X), np.array(Y)
        self.tree = self.build_tree(X, Y, 0) # start building from root at depth 0

    def build_tree(self, X, Y, depth):
        n_samples, n_features = X.shape
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(np.unique(Y)) == 1):
            return self.Node(val=np.mean(Y))  # return a leaf node with mean value

        # Vectorized best split
        best_feat, best_thresh, best_mse = None, None, np.inf
        for f in range(n_features):
            sort_idx = np.argsort(X[:, f])  # Sort feature values
            Xf, Yf = X[sort_idx, f], Y[sort_idx]
            cumsum_y = np.cumsum(Yf)
            cumsum_y2 = np.cumsum(Yf**2)
            n = len(Yf)
            split_points = np.where(Xf[1:] != Xf[:-1])[0] + 1 # Candidate split points are where feature values change
            left_count = split_points
            right_count = n - left_count
            left_mask = (left_count >= self.min_samples_leaf) & (right_count >= self.min_samples_leaf)
            split_points = split_points[left_mask]

            if len(split_points) == 0: # No valid splits for this feature
                continue

            for sp in split_points:
                left_csum, left_csum2 = cumsum_y[sp-1], cumsum_y2[sp-1]
                left_n = sp
                right_csum, right_csum2 = cumsum_y[-1] - left_csum, cumsum_y2[-1] - left_csum2
                right_n = n - sp
                left_mse = (left_csum2 - (left_csum**2)/left_n)/left_n
                right_mse = (right_csum2 - (right_csum**2)/right_n)/right_n
                weighted_mse = (left_mse*left_n + right_mse*right_n)/n

                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feat = f
                    best_thresh = (Xf[sp-1] + Xf[sp])/2 # Split threshold is midpoint

        # if no valid split found
        if best_feat is None:
            return self.Node(val=np.mean(Y))

        # partition data according to the best split
        left_mask = X[:, best_feat] <= best_thresh
        right_mask = ~left_mask
        left = self.build_tree(X[left_mask], y[left_mask], depth+1)
        right = self.build_tree(X[right_mask], y[right_mask], depth+1)
        return self.Node(feat=best_feat, thresh=best_thresh, left=left, right=right)

    def predict(self, X): # prediction using a stack to avoid recursion
        X = np.array(X)
        preds = np.empty(X.shape[0], dtype=float)
        stack = [(np.arange(X.shape[0]), self.tree)]

        while stack:
            idxs, node = stack.pop()
            if node.val is not None:
                preds[idxs] = node.val
                continue
            left_idx = idxs[X[idxs, node.feat] <= node.thresh]
            right_idx = idxs[X[idxs, node.feat] > node.thresh]
            if len(left_idx) > 0:
                stack.append((left_idx, node.left))
            if len(right_idx) > 0:
                stack.append((right_idx, node.right))
        return preds

In [335]:
def grid_search(X_train, Y_train, X_val, Y_val, max_depth_range, min_samples_split_range, min_samples_leaf_range, model_class):
  best_parameters = None # track best hyperparameters
  best_score = float("inf")

  param_grid = itertools.product(max_depth_range, min_samples_split_range, min_samples_leaf_range)

  for depth, split, leaf in param_grid:
      model = model_class(max_depth=depth, min_samples_split=split, min_samples_leaf=leaf)
      model.fit(X_train, Y_train) # Fit the model on training data
      prediction = model.predict(X_val)
      mse = mean_squared_error(Y_val, prediction) # compute mean squared error

      if mse < best_score :
        best_score = mse
        best_parameters = (depth, split, leaf)

  return best_parameters, best_score

In [336]:
best_parameters, best_score = grid_search(X_train, Y_train, X_val, Y_val, range(10, 19), range(7, 13), range(2, 8), DecisionTree)

print(f"Best Parameters : Max Depth={best_parameters[0]}, Min Samples Split={best_parameters[1]}, Min Samples Leaf={best_parameters[2]}") #print the best hyperparameters found
print(f"Best MSE : {best_score}")


NameError: name 'y' is not defined

In [ ]:
scratch_model = DecisionTree(
    max_depth=best_parameters[0],
    min_samples_split=best_parameters[1],
    min_samples_leaf=best_parameters[2]
)
scratch_model.fit(X_train, Y_train) #fit the model on the training data
scratch_pred = scratch_model.predict(X_test)
scratch_mse = mean_squared_error(Y_test, scratch_pred) # Evaluate the model's performance
scratch_r2 = r2_score(Y_test, scratch_pred)

print("Test MSE: ", scratch_mse)
print("Test R2: ", scratch_r2)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(Y_test, scratch_pred)
plt.plot([Y_test.min(), Y_test.max()],
         [Y_test.min(), Y_test.max()],
         "r--", lw=2)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values")
plt.show() # Plot the graph

In [ ]:
best_parameters_scikit, best_score_scikit = grid_search(
    X_train, Y_train, X_val, Y_val,
    range(10, 19), range(7, 13), range(2, 8),
    DecisionTreeRegressor
)

print("Best Hyperparameters (Scikit): ", best_parameters_scikit)
print("Best Validation MSE (Scikit): ", best_score_scikit)

scikit_model = DecisionTreeRegressor(
    max_depth=best_parameters_scikit[0],
    min_samples_split=best_parameters_scikit[1],
    min_samples_leaf=best_parameters_scikit[2],
    random_state=42
)
scikit_model.fit(X_train, Y_train) # create a scikit-learn DecisionTreeRegressor
scikit_pred = scikit_model.predict(X_test)
scikit_mse = mean_squared_error(Y_test, scikit_pred)

print("Scratch Tree Test MSE:", scratch_mse) # print and compare test MSEs of scratch and scikit-learn
print("Scikit-learn Tree Test MSE:", scikit_mse)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(Y_test, scikit_pred)
plt.plot([Y_test.min(), Y_test.max()],
         [Y_test.min(), Y_test.max()],
         "r--", lw=2)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values (Decision Tree)")
plt.show() # Plot the graph

In [ ]:
X_train_df = pd.DataFrame(X_train, columns=X.columns)

plt.figure(figsize=(20, 20))
plot_tree(
    scikit_model,
    filled=True,
    feature_names=X_train_df.columns,
    rounded=True,
    fontsize=10
)

plt.show() # Printing the tree


# We observe from the MSE scores :


*   Scratch Tree Test MSE: 5754.788112277918
*   Scikit-learn Tree Test MSE: 5727.479907594079

The custom Decision Tree works correctly and produces results comparable to scikit-learn’s professional implementation. For real-world tasks, scikit-learn is preferred due to speed and optimizations.
The difference is expected because scikit-learn is highly optimized in C and uses better numerical stability tricks.
